In [1]:
# You will need the ISISDATA area in your environment to get the appropriate kernel paths
import os
os.environ["ISISDATA"] = "/Users/ahibl/working/astro/local_isis_data/"
os.environ["ISISTESTDATA"] = "/Users/ahibl/astro_efs/isis_testData/"
os.environ["ISISROOT"] = "/Users/ahibl/working/astro/ISIS3/build/"
os.environ["ALESPICEROOT"] = "/Users/ahibl/astro_efs/usgs_data/"

import spiceypy as spice
import json
import numpy as np
import pvl

import ale
from ale.base import Driver
from ale.base.label_isis import IsisLabel
from ale.base.data_naif import NaifSpice
from ale.base.type_distortion import RadialDistortion, NoDistortion
from ale.base.type_sensor import Framer, LineScanner
from ale.util import generate_kernels_from_cube
from ale.base.type_sensor import Framer
from ale.base.type_distortion import NoDistortion
from ale.drivers.msl_drivers import MslMastcamPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm

from ale import util
from ale.util import generate_kernels_from_cube
from ale.drivers import AleJsonEncoder, load, loads
from ale.formatters.formatter import to_isd

In [2]:
# Generate kernels from an ISIS spiceinit'd label
cube = "/Users/ahibl/working/astro/img_data/MSI/cubes/ale/ale_spice01.cub"
kernels = generate_kernels_from_cube(cube, expand=True, format_as='list')
kernels
a
# change to desired PDS3 image path
#file_name = '/Users/ahibl/working/astro/img_data/0269ML0011790000106115E01_DRCX.lbl'

# metakernels are furnished when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
#with MslMastcamPds3NaifSpiceDriver(file_name, props={'kernels': ["/Users/ahibl/working/astro/rand/msl_2017_v01.tm"]}) as driver:
    # pass driver instance into formatter function
    #usgscsmString = to_usgscsm(driver)

['/Users/ahibl/working/astro/local_isis_data//base/kernels/spk/de405.bsp',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/tspk/erosephem_1999004_2002181.bsp',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/spk/near_erosorbit_nav_v1.bsp',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/ck/near_2000056_v01.bc',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/fk/eros_fixed.tf',
 '/Users/ahibl/working/astro/local_isis_data//base/kernels/pck/pck00009.tpc',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/ik/msi15.ti',
 '/Users/ahibl/working/astro/local_isis_data//base/kernels/lsk/naif0012.tls',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/sclk/near_171.tsc',
 '/Users/ahibl/working/astro/local_isis_data//near/kernels/iak/msiAddendum002.ti']

In [4]:
with open("/Users/ahibl/working/astro/ISDs/msl_2.json", 'w') as fp:
    json.dump(ale_isd, fp, cls=AleJsonEncoder, indent=2)

NameError: name 'isis_isd' is not defined

In [4]:
# Build new driver in this cell

class MsiIsisLabelNaifSpiceDriver(Framer, IsisLabel, NaifSpice, NoDistortion, Driver):
    
    @property
    def instrument_id(self):
        """
        Returns an instrument id for uniquely identifying the instrument,
        but often also used to be piped into Spice Kernels to acquire
        IKIDS. Therefor they are the same ID that Spice expects in bods2c
        calls. Expect instrument_id to be defined in the IsisLabel mixin.
        This should be a string of the form NEAR EARTH ASTEROID RENDEZVOUS

        Returns
        -------
        : str
          instrument id
        """
        lookup_table = {
        "MSI": "NEAR EARTH ASTEROID RENDEZVOUS"
        }
        print(lookup_table[super().instrument_id])
        return lookup_table[super().instrument_id]

    @property
    def sensor_name(self):
        """
        Returns the name of the instrument

        Returns
        -------
        : str
          instrument name
        """
        return "MULTI-SPECTRAL IMAGER"

    @property
    def sensor_model_version(self):
        """
        Returns ISIS sensor model version

        Returns
        -------
        : int
          ISIS sensor model version
        """
        return 1
    
    # Overridden to remove decimal and 2multiply by a factor of 100 helps remove 2 fields issue cleanly
    @property
    def spacecraft_clock_start_count(self):
        """
        The spacecraft clock start count, frequently used to determine the start time
        of the image.

        Returns
        -------
        : str
          spacecraft clock start count
        """
        if 'SpacecraftClockStartCount' in self.label['IsisCube']['Instrument']:
            
            return str(self.label['IsisCube']['Instrument']['SpacecraftClockStartCount'])
        else:
            return None

    # Overridden to remove decimal and 2multiply by a factor of 100 helps remove 2 fields issue cleanly
    @property
    def spacecraft_clock_stop_count(self):
        """
        The spacecraft clock stop count, frequently used to determine the stop time
        of the image.

        Returns
        -------
        : str
          spacecraft clock stop count
        """
        if 'SpacecraftClockStopCount' in self.label['IsisCube']['Instrument']:
            return str(self.label['IsisCube']['Instrument']['SpacecraftClockStopCount'] * 100).replace(".","")
        else:
            return None

    @property
    def ephemeris_stop_time(self):
        """
        Returns the sum of the starting ephemeris time and the exposure duration.
        Expects ephemeris start time and exposure duration to be defined. These
        should be double precision numbers containing the ephemeris start and
        exposure duration of the image.
        Returns
        -------
        : double
          Ephemeris stop time for an image
        """
        return self.ephemeris_start_time + self.exposure_duration
    
    @property
    def ikid(self):
        """
        Overridden to grab the ikid from the Isis Cube since there is no way to
        obtain this value with a spice bods2c call. Isis sets this value during
        ingestion, based on the original fits file.
        
        Returns
        -------
        : int
          Naif ID used to for identifying the instrument in Spice kernels
        """
        return self.label['IsisCube']['Kernels']['NaifFrameCode']
    

In [7]:
import pprint

# Create ISD from ISIS label
ale_file = "/Users/ahibl/working/astro/img_data/MSI/cubes/ale/ale_spice01.cub"
#unspiced_file = "/Users/ahibl/working/astro/img_data/MSI/ad_1.cub"
#isis_file = "/Users/ahibl/working/astro/img_data/MSI/spicy_ad_1.cub"

with MsiIsisLabelNaifSpiceDriver(ale_file, props = {"kernels" : kernels}) as driver:
    ale_isd = to_isd(driver)

#with MsiIsisLabelNaifSpiceDriver(isis_file, props = {"kernels" : kernels}) as driver:
    #isis_isd = to_isd(driver)

#with MsiIsisLabelNaifSpiceDriver(unspiced_file, props = {"kernels" : kernels}) as driver:
    #unspicy_isd = to_isd(driver)
    
#isds = [ale_isd, isis_isd, unspicy_isd]

#pprint.pprint(isds[0])

#print("\n\n")

#pprint.pprint(isds[1])

#print("\n\n")

#pprint.pprint(isds[2])

In [12]:
# with open("/Users/ahibl/working/astro/ISDs/msl_1.json", 'w') as fp:
#     json.dump(usgscsmString, fp, cls=AleJsonEncoder, indent=2)

In [ ]:
file = "/Users/ahibl/working/astro/ISIS3/isis/tests/data/near/msi2isis/test.cub"

with MsiIsisLabelNaifSpiceDriver(file, props = {"kernels" : kernels}) as driver:
    #test area

In [8]:
# Write ISD to file
with open("/Users/ahibl/working/astro/ISDs/msi_aled.json", 'w') as fp:
    json.dump(ale_isd, fp, cls=AleJsonEncoder, indent=2)
#with open("/Users/ahibl/working/astro/ISDs/msi_isis.json", 'w') as fp:
#    json.dump(isis_isd, fp, cls=AleJsonEncoder, indent=2)
#with open("/Users/ahibl/working/astro/ISDs/msi_unspiced.json", 'w') as fp:
#    json.dump(unspicy_isd, fp, cls=AleJsonEncoder, indent=2)

In [3]:
file = '/Users/ahibl/working/astro/img_data/MSI/labels/ale_spice01.lbl'
ale.load(file, "ale")

Banana False False
[<class 'ale.drivers.msi_drivers.MsiIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.osirisrex_drivers.OsirisRexCameraIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.galileo_drivers.GalileoSsiIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.clipper_drivers.ClipperEISNACFCIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.clipper_drivers.ClipperEISNACPBIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.clipper_drivers.ClipperEISWACFCIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.clipper_drivers.ClipperEISWACPBIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.dawn_drivers.DawnFcPds3NaifSpiceDriver'>, <class 'ale.drivers.co_drivers.CassiniIssIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.co_drivers.CassiniIssPds3LabelNaifSpiceDriver'>, <class 'ale.drivers.msl_drivers.MslMastcamPds3NaifSpiceDriver'>, <class 'ale.drivers.lro_drivers.LroLrocNacIsisLabelNaifSpiceDriver'>, <class 'ale.drivers.lro_drivers.LroLrocNacPds3LabelNaifSpiceDriver'>, <class 'ale.drivers.lro_drivers.LroLrocWa

Traceback (most recent call last):
  File "/Users/ahibl/working/astro/ale/ale/drivers/__init__.py", line 153, in load
    res = driver(label, props=props, parsed_label=parsed_label)
  File "/Users/ahibl/working/astro/ale/ale/base/base.py", line 26, in __init__
    self._props = json.loads(props)
  File "/Users/ahibl/homebrew/Caskroom/miniconda/base/envs/ale-dvlp/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/ahibl/homebrew/Caskroom/miniconda/base/envs/ale-dvlp/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/ahibl/homebrew/Caskroom/miniconda/base/envs/ale-dvlp/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/Users/ahibl/working/astro/ale/ale/drivers/__init__.py", lin

Exception: No Such Driver for Label

In [9]:
with open(file) as f:
    print(f.read())

Object = IsisCube
  Object = Core
    StartByte   = 65537
    Format      = Tile
    TileSamples = 179
    TileLines   = 412

    Group = Dimensions
      Samples = 537
      Lines   = 412
      Bands   = 1
    End_Group

    Group = Pixels
      Type       = Real
      ByteOrder  = Lsb
      Base       = 0.0
      Multiplier = 1.0
    End_Group
  End_Object

  Group = Instrument
    SpacecraftName                    = "NEAR EARTH ASTEROID RENDEZVOUS"
    InstrumentId                      = MSI
    TargetName                        = EROS
    StartTime                         = 2000-02-25T05:16:12.656
    StopTime                          = 2000-02-25T05:16:12.745
    SpacecraftClockStartCount         = 126865998830
    SpacecraftClockStopCount          = 126865998919
    ExposureDuration                  = 89.00 <ms>
    OriginalSpacecraftClockStartCount = 126865998.830
    OriginalSpacecraftClockStopCount  = 126865998.919
    DpuDeckTemperature                = 286.50 <K>
  End_Group